In [1]:
from rscube.aws import upload_file_to_s3
from pathlib import Path
import geopandas as gpd
import pandas as pd
from tqdm import tqdm

# Upload Directory

Uses *relative* path to generate prefix and key for s3 bucket. Assumes metadata.json.

Probably should re-write this so no side-effects.

In [10]:
class_directories = list(Path('classification_outputs_peckel').glob('*/'))
class_directories = list(filter(lambda x: '.DS_Store' not in x.name, class_directories))
class_directories

[PosixPath('classification_outputs_peckel/20211011_155455_52_2262'),
 PosixPath('classification_outputs_peckel/20210925_191708_94_2408'),
 PosixPath('classification_outputs_peckel/20211014_152910_49_2459'),
 PosixPath('classification_outputs_peckel/20210926_072414_71_2233'),
 PosixPath('classification_outputs_peckel/20211003_161639_91_241d'),
 PosixPath('classification_outputs_peckel/20210914_051809_10_2455'),
 PosixPath('classification_outputs_peckel/20210901_175205_71_2426'),
 PosixPath('classification_outputs_peckel/20211028_045455_02_2459'),
 PosixPath('classification_outputs_peckel/20210906_183639_52_240c'),
 PosixPath('classification_outputs_peckel/20210924_082025_48_2424'),
 PosixPath('classification_outputs_peckel/20211027_151445_21_2231'),
 PosixPath('classification_outputs_peckel/20211030_142613_41_227b'),
 PosixPath('classification_outputs_peckel/20211010_080351_47_2458'),
 PosixPath('classification_outputs_peckel/20211005_135201_42_227e')]

In [11]:
def upload_directory(dir_path: Path) -> list:
    df_meta = gpd.GeoDataFrame.from_file(dir_path / 'metadata.geojson')
    
    BUCKET = df_meta.s3_bucket[0]
    PREFIX = df_meta.s3_prefix[0]
    assert(PREFIX == str(dir_path))
    
    paths = list(dir_path.glob('*'))
    paths = list(filter(lambda x: '.DS_Store' not in x.name, paths))
    paths = list(filter(lambda x: 'cropped' not in x.name, paths))
    
    upload_file_to_s3_p = lambda path: upload_file_to_s3(path, BUCKET, PREFIX)

    return list(map(upload_file_to_s3_p, tqdm(paths, desc=f'Uploading files in {dir_path.name}')))

In [12]:
list(map(upload_directory, tqdm(class_directories, desc='Uploading Directories')))

Uploading files in 20211011_155455_52_2262:   0%|   | 0/7 [00:00<?, ?it/s]
Uploading files in 20211011_155455_52_2262:  14%|▏| 1/7 [00:02<00:15,  2.5
Uploading files in 20211011_155455_52_2262:  29%|▎| 2/7 [00:02<00:05,  1.1
Uploading files in 20211011_155455_52_2262:  43%|▍| 3/7 [00:03<00:03,  1.0
Uploading files in 20211011_155455_52_2262:  57%|▌| 4/7 [03:11<03:43, 74.6
Uploading files in 20211011_155455_52_2262:  71%|▋| 5/7 [03:11<01:35, 47.9
Uploading files in 20211011_155455_52_2262:  86%|▊| 6/7 [03:11<00:31, 31.7
Uploading files in 20211011_155455_52_2262: 100%|█| 7/7 [03:12<00:00, 27.4
Uploading files in 20210925_191708_94_2408:   0%|   | 0/8 [00:00<?, ?it/s]
Uploading files in 20210925_191708_94_2408:  12%|▏| 1/8 [00:00<00:03,  2.2
Uploading files in 20210925_191708_94_2408:  25%|▎| 2/8 [00:00<00:01,  3.6
Uploading files in 20210925_191708_94_2408:  38%|▍| 3/8 [00:01<00:01,  2.9
Uploading files in 20210925_191708_94_2408:  50%|▌| 4/8 [00:01<00:01,  2.5
Uploading files in 202109

[['opera-calval-dswx-jupyter/classification_outputs_peckel/20211011_155455_52_2262/training_data_reprojected_20211011_155455_52_2262.tif',
  'opera-calval-dswx-jupyter/classification_outputs_peckel/20211011_155455_52_2262/metadata.geojson',
  'opera-calval-dswx-jupyter/classification_outputs_peckel/20211011_155455_52_2262/occurrence_20211011_155455_52_2262.tif',
  'opera-calval-dswx-jupyter/classification_outputs_peckel/20211011_155455_52_2262/superpixel_means_scale-0_20211011_155455_52_2262.tif',
  'opera-calval-dswx-jupyter/classification_outputs_peckel/20211011_155455_52_2262/classification_20211011_155455_52_2262.tif',
  'opera-calval-dswx-jupyter/classification_outputs_peckel/20211011_155455_52_2262/classification_HAND_UPDATES_20211011_155455_52_2262.tif',
  'opera-calval-dswx-jupyter/classification_outputs_peckel/20211011_155455_52_2262/20211011_155455_52_2262.ipynb'],
 ['opera-calval-dswx-jupyter/classification_outputs_peckel/20210925_191708_94_2408/20210925_191708_94_2408.ipynb

# Aggregate Table

In [13]:
import boto3

s3client = boto3.client('s3')
BUCKET_NAME = 'opera-calval-dswx-jupyter'
objects = s3client.list_objects(Bucket=BUCKET_NAME)['Contents']
objects[:5]

[{'Key': 'classification_outputs_peckel/20210901_175205_71_2426/20210901_175205_71_2426.ipynb',
  'LastModified': datetime.datetime(2022, 7, 22, 23, 9, 28, tzinfo=tzutc()),
  'ETag': '"183b31a590b147677e17586281b2b219"',
  'Size': 494058,
  'StorageClass': 'STANDARD',
  'Owner': {'DisplayName': 'opera-cal-val-aws',
   'ID': '8f0d53d2be7bf0d9605dc36d5e133a729cf214a50ea5eb5618b53d47fdab24f2'}},
 {'Key': 'classification_outputs_peckel/20210901_175205_71_2426/classification_20210901_175205_71_2426.tif',
  'LastModified': datetime.datetime(2022, 7, 22, 23, 13, 47, tzinfo=tzutc()),
  'ETag': '"187bda92b57c08225c9a542106499da1"',
  'Size': 757603,
  'StorageClass': 'STANDARD',
  'Owner': {'DisplayName': 'opera-cal-val-aws',
   'ID': '8f0d53d2be7bf0d9605dc36d5e133a729cf214a50ea5eb5618b53d47fdab24f2'}},
 {'Key': 'classification_outputs_peckel/20210901_175205_71_2426/classification_HAND_UPDATES_20210901_175205_71_2426.tif',
  'LastModified': datetime.datetime(2022, 7, 22, 23, 13, 45, tzinfo=tzut

In [14]:
objs_meta = list(filter(lambda obj: 'metadata.geojson' == obj['Key'].split('/')[-1], objects))
objs_meta

[{'Key': 'classification_outputs_peckel/20210901_175205_71_2426/metadata.geojson',
  'LastModified': datetime.datetime(2022, 7, 22, 23, 13, 42, tzinfo=tzutc()),
  'ETag': '"1e6e11c21762a1eb061e86f6b49c4c4f"',
  'Size': 1555,
  'StorageClass': 'STANDARD',
  'Owner': {'DisplayName': 'opera-cal-val-aws',
   'ID': '8f0d53d2be7bf0d9605dc36d5e133a729cf214a50ea5eb5618b53d47fdab24f2'}},
 {'Key': 'classification_outputs_peckel/20210906_183639_52_240c/metadata.geojson',
  'LastModified': datetime.datetime(2022, 7, 22, 23, 18, 4, tzinfo=tzutc()),
  'ETag': '"74a839011d2fb04e36cd1998249f3700"',
  'Size': 1549,
  'StorageClass': 'STANDARD',
  'Owner': {'DisplayName': 'opera-cal-val-aws',
   'ID': '8f0d53d2be7bf0d9605dc36d5e133a729cf214a50ea5eb5618b53d47fdab24f2'}},
 {'Key': 'classification_outputs_peckel/20210914_051809_10_2455/metadata.geojson',
  'LastModified': datetime.datetime(2022, 7, 22, 23, 5, 21, tzinfo=tzutc()),
  'ETag': '"e2730dc26c6c094ceadcede509078034"',
  'Size': 1552,
  'StorageCla

In [15]:
def read_one(obj):
    key = obj['Key']
    return gpd.read_file(f's3://{BUCKET_NAME}/{key}')

In [16]:
dfs = list(map(read_one, objs_meta))
df_meta_all = pd.concat(dfs, axis=0)
df_meta_all

,feature_list,feature_importances,accuracy,s3_bucket,s3_prefix,s3_keys,chip_id,strata,planet_id,geometry
0,"pixels (rgb) ,mean_superpixel (rgb for 3 segs)...","0.175982,0.041749,0.013511,0.163051,0.061744,0...","{'not_water': {'not_water': 163010, 'water': 2...",opera-calval-dswx-jupyter,classification_outputs_peckel/20210901_175205_...,"20210901_175205_71_2426.ipynb,chip_cropped_202...",115,4,20210901_175205_71_2426,"POLYGON ((612168.818 6967451.512, 623464.758 6..."
0,"pixels (rgb) ,mean_superpixel (rgb for 3 segs)...","0.052740,0.021781,0.064353,0.069581,0.022370,0...","{'not_water': {'not_water': 27144, 'water': 2}...",opera-calval-dswx-jupyter,classification_outputs_peckel/20210906_183639_...,"20210906_183639_52_240c.ipynb,chip_cropped_202...",16,2,20210906_183639_52_240c,"POLYGON ((518606.228 4463298.082, 537382.421 4..."
0,"pixels (rgb) ,mean_superpixel (rgb for 3 segs)...","0.097939,0.021023,0.016045,0.105957,0.033215,0...","{'not_water': {'not_water': 52374, 'water': 17...",opera-calval-dswx-jupyter,classification_outputs_peckel/20210914_051809_...,"20210914_051809_10_2455.ipynb,chip_cropped_202...",109,1,20210914_051809_10_2455,"POLYGON ((484725.808 4023892.372, 490804.086 4..."
0,"pixels (rgb) ,mean_superpixel (rgb for 3 segs)...","0.136184,0.001254,0.000614,0.134204,0.006352,0...","{'not_water': {'not_water': 24186, 'water': 5}...",opera-calval-dswx-jupyter,classification_outputs_peckel/20210924_082025_...,"20210924_082025_48_2424.ipynb,chip_cropped_202...",70,4,20210924_082025_48_2424,"POLYGON ((657224.012 8984934.146, 657143.906 8..."
0,"pixels (rgb) ,mean_superpixel (rgb for 3 segs)...","0.066320,0.023198,0.045239,0.076251,0.031792,0...","{'not_water': {'not_water': 76170, 'water': 2}...",opera-calval-dswx-jupyter,classification_outputs_peckel/20210925_191708_...,".DS_Store,20210925_191708_94_2408.ipynb,chip_c...",77,3,20210925_191708_94_2408,"POLYGON ((497825.315 6424257.440, 499704.124 6..."
0,"pixels (rgb) ,mean_superpixel (rgb for 3 segs)...","0.086234,0.043117,0.006728,0.064763,0.061405,0...","{'not_water': {'not_water': 28325, 'water': 8}...",opera-calval-dswx-jupyter,classification_outputs_peckel/20210926_072414_...,"20210926_072414_71_2233.ipynb,chip_cropped_202...",28,4,20210926_072414_71_2233,"POLYGON ((575355.185 8208812.186, 575286.456 8..."
0,"pixels (rgb) ,mean_superpixel (rgb for 3 segs)...","0.158141,0.006974,0.009146,0.108247,0.009211,0...","{'not_water': {'not_water': 58529, 'water': 1}...",opera-calval-dswx-jupyter,classification_outputs_peckel/20211003_161639_...,"20211003_161639_91_241d.ipynb,chip_cropped_202...",42,3,20211003_161639_91_241d,"POLYGON ((222530.752 3668526.777, 241571.451 3..."
0,"pixels (rgb) ,mean_superpixel (rgb for 3 segs)...","0.120593,0.037206,0.013532,0.121891,0.063324,0...",{'not_water': {'not_water': 72374}},opera-calval-dswx-jupyter,classification_outputs_peckel/20211005_135201_...,"20211005_135201_42_227e.ipynb,chip_cropped_202...",116,1,20211005_135201_42_227e,"POLYGON ((737284.864 6172710.343, 744282.479 6..."
0,"pixels (rgb) ,mean_superpixel (rgb for 3 segs)...","0.036724,0.009336,0.034557,0.119451,0.050768,0...","{'not_water': {'not_water': 31103, 'water': 0}...",opera-calval-dswx-jupyter,classification_outputs_peckel/20211010_080351_...,"20211010_080351_47_2458.ipynb,chip_cropped_202...",87,2,20211010_080351_47_2458,"POLYGON ((547375.493 6063882.259, 558454.187 6..."
0,"pixels (rgb) ,mean_superpixel (rgb for 3 segs)...","0.110703,0.017060,0.021436,0.111978,0.015316,0...","{'not_water': {'not_water': 111049, 'water': 4...",opera-calval-dswx-jupyter,classification_outputs_peckel/20211011_155455_...,"20211011_155455_52_2262.ipynb,chip_cropped_202...",91,3,20211011_155455_52_2262,"POLYGON ((773264.766 1812690.844, 780398.977 1..."


In [17]:
TABLE_NAME = f'dswx_classification_rf_peckel.geojson'
df_meta_all.to_file(f's3://{BUCKET_NAME}/{TABLE_NAME}', driver='GeoJSON')

In [18]:
df_meta_all

,feature_list,feature_importances,accuracy,s3_bucket,s3_prefix,s3_keys,chip_id,strata,planet_id,geometry
0,"pixels (rgb) ,mean_superpixel (rgb for 3 segs)...","0.175982,0.041749,0.013511,0.163051,0.061744,0...","{'not_water': {'not_water': 163010, 'water': 2...",opera-calval-dswx-jupyter,classification_outputs_peckel/20210901_175205_...,"20210901_175205_71_2426.ipynb,chip_cropped_202...",115,4,20210901_175205_71_2426,"POLYGON ((612168.818 6967451.512, 623464.758 6..."
0,"pixels (rgb) ,mean_superpixel (rgb for 3 segs)...","0.052740,0.021781,0.064353,0.069581,0.022370,0...","{'not_water': {'not_water': 27144, 'water': 2}...",opera-calval-dswx-jupyter,classification_outputs_peckel/20210906_183639_...,"20210906_183639_52_240c.ipynb,chip_cropped_202...",16,2,20210906_183639_52_240c,"POLYGON ((518606.228 4463298.082, 537382.421 4..."
0,"pixels (rgb) ,mean_superpixel (rgb for 3 segs)...","0.097939,0.021023,0.016045,0.105957,0.033215,0...","{'not_water': {'not_water': 52374, 'water': 17...",opera-calval-dswx-jupyter,classification_outputs_peckel/20210914_051809_...,"20210914_051809_10_2455.ipynb,chip_cropped_202...",109,1,20210914_051809_10_2455,"POLYGON ((484725.808 4023892.372, 490804.086 4..."
0,"pixels (rgb) ,mean_superpixel (rgb for 3 segs)...","0.136184,0.001254,0.000614,0.134204,0.006352,0...","{'not_water': {'not_water': 24186, 'water': 5}...",opera-calval-dswx-jupyter,classification_outputs_peckel/20210924_082025_...,"20210924_082025_48_2424.ipynb,chip_cropped_202...",70,4,20210924_082025_48_2424,"POLYGON ((657224.012 8984934.146, 657143.906 8..."
0,"pixels (rgb) ,mean_superpixel (rgb for 3 segs)...","0.066320,0.023198,0.045239,0.076251,0.031792,0...","{'not_water': {'not_water': 76170, 'water': 2}...",opera-calval-dswx-jupyter,classification_outputs_peckel/20210925_191708_...,".DS_Store,20210925_191708_94_2408.ipynb,chip_c...",77,3,20210925_191708_94_2408,"POLYGON ((497825.315 6424257.440, 499704.124 6..."
0,"pixels (rgb) ,mean_superpixel (rgb for 3 segs)...","0.086234,0.043117,0.006728,0.064763,0.061405,0...","{'not_water': {'not_water': 28325, 'water': 8}...",opera-calval-dswx-jupyter,classification_outputs_peckel/20210926_072414_...,"20210926_072414_71_2233.ipynb,chip_cropped_202...",28,4,20210926_072414_71_2233,"POLYGON ((575355.185 8208812.186, 575286.456 8..."
0,"pixels (rgb) ,mean_superpixel (rgb for 3 segs)...","0.158141,0.006974,0.009146,0.108247,0.009211,0...","{'not_water': {'not_water': 58529, 'water': 1}...",opera-calval-dswx-jupyter,classification_outputs_peckel/20211003_161639_...,"20211003_161639_91_241d.ipynb,chip_cropped_202...",42,3,20211003_161639_91_241d,"POLYGON ((222530.752 3668526.777, 241571.451 3..."
0,"pixels (rgb) ,mean_superpixel (rgb for 3 segs)...","0.120593,0.037206,0.013532,0.121891,0.063324,0...",{'not_water': {'not_water': 72374}},opera-calval-dswx-jupyter,classification_outputs_peckel/20211005_135201_...,"20211005_135201_42_227e.ipynb,chip_cropped_202...",116,1,20211005_135201_42_227e,"POLYGON ((737284.864 6172710.343, 744282.479 6..."
0,"pixels (rgb) ,mean_superpixel (rgb for 3 segs)...","0.036724,0.009336,0.034557,0.119451,0.050768,0...","{'not_water': {'not_water': 31103, 'water': 0}...",opera-calval-dswx-jupyter,classification_outputs_peckel/20211010_080351_...,"20211010_080351_47_2458.ipynb,chip_cropped_202...",87,2,20211010_080351_47_2458,"POLYGON ((547375.493 6063882.259, 558454.187 6..."
0,"pixels (rgb) ,mean_superpixel (rgb for 3 segs)...","0.110703,0.017060,0.021436,0.111978,0.015316,0...","{'not_water': {'not_water': 111049, 'water': 4...",opera-calval-dswx-jupyter,classification_outputs_peckel/20211011_155455_...,"20211011_155455_52_2262.ipynb,chip_cropped_202...",91,3,20211011_155455_52_2262,"POLYGON ((773264.766 1812690.844, 780398.977 1..."
